In [10]:
import os
os.environ['PYTENSOR_FLAGS'] = 'mode=FAST_COMPILE,optimizer=fast_compile,floatX=float32,cxx='

import numpy as np
import pymc as pm
import pytensor.tensor as pt

In this notebook, we demonstrate the initialization capability of PyMC's ADVI.

We will fit a simple 1d normal distribution using ADVI and compare the initializations.


In [26]:

with pm.Model() as init_model:
    # Define priors
    mu = pm.Normal("mu", 0, 1)
    likelihood = pm.Normal('y', mu=mu, sigma=1, observed=None)
    advi = pm.ADVI(start={'mu': 1.0})
    init_fit = advi.fit(0)

with pm.Model() as default_model:
    # Define priors
    mu = pm.Normal("mu", 0, 1)
    likelihood = pm.Normal('y', mu=mu, sigma=1, observed=None)
    default_fit = pm.fit(0, method=pm.ADVI())



Output()

Initialization only


Output()

Initialization only


In [27]:
# we figure out which index mu is - just to make sure we are looking at the correct value
default_mu_idx = [var.name for var in default_model.unobserved_RVs].index("mu")
init_mu_idx = [var.name for var in init_model.unobserved_RVs].index("mu")
default_mu_idx, init_mu_idx

(0, 0)

In [28]:
# This is what the default initialized to
default_fit.mean.eval()[default_mu_idx], default_fit.std.eval()[default_mu_idx]

(np.float32(0.0), np.float32(0.6931472))

In [29]:
# This is what the init initialized to, 
# when we told it to use start={'mu': 1.0}
init_fit.mean.eval()[init_mu_idx], init_fit.std.eval()[init_mu_idx]

(np.float32(1.0), np.float32(0.6931472))

Now, let's use a Beta model to determine whether this initialization is changing the constrained value in $(0, 1)$ or unconstrained mean in $\mathbb{R}$.

We will sample from the posterior and take the empirical mean and std and compare it to the initial parameter value of $0.5$.

In [77]:
with pm.Model() as beta_model:
    # Define priors
    mu = pm.Beta("theta", 2, 5)
    likelihood = pm.Binomial('obs', n=10, p=mu, observed=2)
    advi = pm.ADVI(start={'theta': 0.5})
    beta_fit = advi.fit(0)

beta_samples = beta_fit.sample(100_000)


Output()

Initialization only


In [78]:
print(f"The empirical mean and std of the samples drawn directly from the posterior are:\n {float(beta_samples.posterior.theta.mean())} and {float(beta_samples.posterior.theta.std())}")


The empirical mean and std of the samples drawn directly from the posterior are:
 0.4999375641345978 and 0.15653567016124725


We can see from the empirical mean that the initialized value initialized the constrained mean.

In [64]:
beta_fit